In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
from sklearn import metrics
from tensorflow import keras
from sklearn.metrics.pairwise import cosine_similarity
import gensim.downloader
from tqdm import tqdm
import numpy as np
import string
import pickle
import csv 
import re

In [2]:
np.random.seed(0)

# モデルの保存名に使用
import datetime
t_delta = datetime.timedelta(hours=9)
JST = datetime.timezone(t_delta, 'JST')
now = datetime.datetime.now(JST)

In [3]:
# parameters
SAVED_MODEL = "../model/Proposed-Word2Vec-TextCNN_"+str(now.strftime('%Y%m%d%H%M%S'))
THRESHOLD = 0.05
MAXLEN_GET_PSEUDO = 3000
EPOCH = 5
BATCH_SIZE = 64
MAX_LEN = 128
FILTERS = 100
FILTER_SIZES = [3,4,5]

In [4]:
print(SAVED_MODEL)

../model/Proposed-Word2Vec-TextCNN_20221201115548


In [5]:
# 前処理
def preprocessing(text):
    # 括弧内文章の削除
    text = re.sub(r'\(.*\)',' ',text)
    text = re.sub(r'\[.*\]',' ',text)
    text = re.sub(r'\<.*\>',' ',text)
    text = re.sub(r'\{.*\}',' ',text)
    # 記号文字の削除
    text = text.translate(str.maketrans('','',string.punctuation))
    # スペースの調整
    text = re.sub(r'\s+',' ',text)
    return text

In [6]:
# 20 newsgroups datasets
from sklearn.datasets import fetch_20newsgroups
newsgroups = fetch_20newsgroups(subset="all")
newsgroups_datasets = list()

# # example ------------------------------------------------
# for texts in tqdm(newsgroups.data[:1000]):
#   texts = texts.split("\n\n")
#   texts = " ".join(texts[1:])
#   newsgroups_datasets.append(preprocessing(texts))
# # --------------------------------------------------------

for texts in tqdm(newsgroups.data):
  texts = texts.split("\n\n")
  texts = " ".join(texts[1:])
  newsgroups_datasets.append(preprocessing(texts))

100%|██████████| 18846/18846 [00:01<00:00, 17731.09it/s]


In [7]:
# yahoo topic datasets
with open('../data/topic/train_pu_half_v0.txt','r',encoding='utf-8') as f:
    texts_v0 = f.read()
with open('../data/topic/train_pu_half_v1.txt','r',encoding='utf-8') as f:
    texts_v1 = f.read()
texts = texts_v0 + texts_v1
topic_datasets = list()

# # example ----------------------------------------------
# for label_text in tqdm(texts.splitlines()[:1000]):
#   _, text = label_text.split("\t")
#   topic_datasets.append(preprocessing(text))
# # -------------------------------------------------------

for label_text in tqdm(texts.splitlines()):
  _, text = label_text.split("\t")
  topic_datasets.append(preprocessing(text))

100%|██████████| 1300000/1300000 [00:28<00:00, 45566.90it/s]


In [8]:
# reuters datasets
with open("../data/reuter/sourceall.txt", "r", encoding="utf-8") as f:
  reuter = f.read().split("\n")[:-1]

# # example -------------------
# import random
# reuter = random.sample(reuter, 1000)
# #----------------------------

reuters_datasets = list()
for label_text in tqdm(reuter):
  _, text = label_text.split("\t")
  reuters_datasets.append(preprocessing(text))

100%|██████████| 762027/762027 [00:30<00:00, 24952.63it/s]


In [9]:
# dbpedia datasets train
with open('../data/dbpedia_csv/train.csv','r',encoding='utf-8') as f:
    reader = [r for r in csv.reader(f)]
    
# # example -------------------
# import random
# reader = random.sample(reader, 1000)
# #----------------------------

dbpedia_train_datasets = list()
for _, auth, text in tqdm(reader):
    text = text.replace(auth,'')
    dbpedia_train_datasets.append(preprocessing(text))

100%|██████████| 560000/560000 [00:08<00:00, 66806.83it/s]


In [10]:
datasets_texts = newsgroups_datasets + topic_datasets + reuters_datasets + dbpedia_train_datasets

In [11]:
# dbpedia classes
with open("../data/dbpedia_csv/classes.txt", "r", encoding="utf-8") as f:
  classes = f.read().splitlines()

In [12]:
# load test data
# dbpedia datasets test
with open('../data/dbpedia_csv/test.csv','r',encoding='utf-8') as f:
    reader = [r for r in csv.reader(f)]
    
# # example -------------------
# import random
# reader = random.sample(reader, 1000)
# #----------------------------

test_texts = list()
test_labels = list()
for labels, auth, text in tqdm(reader):
    text = text.replace(auth,'')
    test_texts.append(preprocessing(text))
    test_labels.append(int(labels)-1)

100%|██████████| 70000/70000 [00:01<00:00, 64276.97it/s]


# Choice method

In [13]:
word2vec = gensim.downloader.load('word2vec-google-news-300')

In [14]:
# 文章をベクトルに変換
# 文章内に複数同じ単語が出現する場合、1度だけ使用する
def w2v_avg_vector(sentence):
  vector = np.zeros((300,), dtype="float32")
  count = 0
  used_word = list()
  for word in sentence.split():
    if word not in used_word:
      used_word.append(word)
      try:
        vector = np.add(vector, word2vec[word])
        count += 1
      except:
        pass
    if count >= MAX_LEN:
      break
  if count > 0:
    vector = np.divide(vector, count)
  return vector

In [15]:
classes_vector = [w2v_avg_vector(cls) for cls in classes]

# # # クラス名にwordnetを適用
# classes_vector = list()
# for cls in classes:
#   wordnet_sequence = list()
#   for word in cls.split():
#     if wordnet.synsets(word):
#       wordnet_sequence.append(wordnet.synsets(word)[0].definition())
#   classes_vector.append(w2v_avg_vector(preprocessing(" ".join(wordnet_sequence))))

In [16]:
# 情報源領域の文章と各クラスの類似度を計算し、上位2クラスの差が閾値を超えた場合、1位のクラスの学習データとする
diff_datasets = {i:[] for i in range(len(classes))}
for texts in tqdm(datasets_texts):
  texts_vector = w2v_avg_vector(texts)
  similarity = cosine_similarity([texts_vector], classes_vector)[0]
  sim_argsorted = np.argsort(similarity)
  diff = similarity[sim_argsorted[-1]] - similarity[sim_argsorted[-2]]
  if diff >= THRESHOLD:
    diff_datasets[sim_argsorted[-1]].append((diff, texts))

# 上位2クラスの差が閾値を超えた文章の中で、その差が大きいものから順に各クラスの疑似ラベル付きデータの数が同じになるように選択
pseudo_texts = list()
pseudo_labels = list()
for i in range(len(classes)):
  sorted_diff_data = sorted(diff_datasets[i], reverse=True)[:MAXLEN_GET_PSEUDO]
  pseudo_texts.extend([i[1] for i in sorted_diff_data])
  pseudo_labels.extend([i]*len(sorted_diff_data))

100%|██████████| 2640873/2640873 [12:03<00:00, 3652.66it/s]


In [17]:
print("Number of all selected data")
for i in diff_datasets:
  print(classes[i][:3]+". : "+str(len(diff_datasets[i])))

Number of all selected data
Com. : 71598
Edu. : 36764
Art. : 7625
Ath. : 17758
Off. : 86410
Mea. : 204570
Bui. : 9172
Nat. : 37282
Vil. : 34984
Ani. : 4312
Pla. : 13165
Alb. : 39291
Fil. : 23275
Wri. : 23976


# Train

In [18]:
tokenizer = keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(pseudo_texts)
tokenizer.fit_on_texts(test_texts)
embedding_matrix = np.zeros((max(tokenizer.word_index.values())+1, word2vec.vector_size))
for word, i in tokenizer.word_index.items():
    if word in word2vec:
        embedding_matrix[i] = word2vec[word]

sequences = tokenizer.texts_to_sequences(pseudo_texts)
x_train = keras.preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_LEN, padding='post')
y_train = np.array(keras.utils.to_categorical(pseudo_labels))

sequences = tokenizer.texts_to_sequences(test_texts)
x_test = keras.preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_LEN, padding='post')
y_test = np.array(keras.utils.to_categorical(test_labels))

In [19]:
inputs = keras.layers.Input(shape=(MAX_LEN),dtype='float32')
embedding = keras.layers.Embedding(input_dim=len(embedding_matrix),output_dim=len(embedding_matrix[0]),weights=[embedding_matrix],trainable=False, mask_zero=True,name='embedding')(inputs)
conv1 = keras.layers.Conv1D(filters=FILTERS, kernel_size=FILTER_SIZES[0], kernel_initializer='normal', activation='relu')(embedding)
conv2 = keras.layers.Conv1D(filters=FILTERS, kernel_size=FILTER_SIZES[1], kernel_initializer='normal', activation='relu')(embedding)
conv3 = keras.layers.Conv1D(filters=FILTERS, kernel_size=FILTER_SIZES[2], kernel_initializer='normal', activation='relu')(embedding)
pool1 = keras.layers.MaxPooling1D(pool_size=int(conv1.shape[1]),strides=1)(conv1)
pool2 = keras.layers.MaxPooling1D(pool_size=int(conv2.shape[1]),strides=1)(conv2)
pool3 = keras.layers.MaxPooling1D(pool_size=int(conv3.shape[1]),strides=1)(conv3)
x = keras.layers.concatenate([pool1, pool2, pool3])
x = keras.layers.Flatten()(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(100,activation='relu')(x)
output = keras.layers.Dense(units=14, activation='softmax')(x)
model = keras.models.Model(inputs, output)
opt = keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=True)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [20]:
# モデルの学習
result = model.fit(x=x_train, y=y_train, epochs=EPOCH, batch_size=BATCH_SIZE, validation_data=(x_test,y_test))

Epoch 1/5
657/657 [==============================] - 9s 8ms/step - loss: 1.1958 - accuracy: 0.6907 - val_loss: 1.0671 - val_accuracy: 0.6771
Epoch 2/5
657/657 [==============================] - 5s 7ms/step - loss: 0.2434 - accuracy: 0.9481 - val_loss: 0.9709 - val_accuracy: 0.6935
Epoch 3/5
657/657 [==============================] - 5s 7ms/step - loss: 0.1321 - accuracy: 0.9696 - val_loss: 0.9977 - val_accuracy: 0.6935
Epoch 4/5
657/657 [==============================] - 5s 7ms/step - loss: 0.0956 - accuracy: 0.9763 - val_loss: 1.0939 - val_accuracy: 0.6802
Epoch 5/5
657/657 [==============================] - 5s 7ms/step - loss: 0.0767 - accuracy: 0.9809 - val_loss: 1.0891 - val_accuracy: 0.6888


In [21]:
model.save(SAVED_MODEL)
with open(SAVED_MODEL+"/tokenizer.pickle", 'wb') as handle:
  pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: ../model/Proposed-Word2Vec-TextCNN_20221201115548/assets


INFO:tensorflow:Assets written to: ../model/Proposed-Word2Vec-TextCNN_20221201115548/assets


# Test

In [22]:
model = keras.models.load_model(SAVED_MODEL)
with open(SAVED_MODEL+"/tokenizer.pickle", 'rb') as handle:
  tokenizer = pickle.load(handle)

In [23]:
pred = model.predict(x_test)

2188/2188 [==============================] - 3s 1ms/step


In [24]:
y_test = test_labels
y_pred = [np.argmax(i) for i in pred]

target_names = [c[:3]+"." for c in classes]

rep = metrics.classification_report(y_test, y_pred, target_names=target_names, digits=3)
print(rep)

              precision    recall  f1-score   support

        Com.      0.848     0.559     0.674      5000
        Edu.      0.702     0.885     0.783      5000
        Art.      0.827     0.536     0.650      5000
        Ath.      0.955     0.862     0.906      5000
        Off.      0.644     0.796     0.712      5000
        Mea.      0.674     0.462     0.549      5000
        Bui.      0.703     0.675     0.689      5000
        Nat.      0.332     0.893     0.484      5000
        Vil.      0.902     0.956     0.928      5000
        Ani.      0.668     0.156     0.253      5000
        Pla.      0.790     0.428     0.556      5000
        Alb.      0.876     0.955     0.914      5000
        Fil.      0.869     0.739     0.799      5000
        Wri.      0.628     0.739     0.679      5000

    accuracy                          0.689     70000
   macro avg      0.744     0.689     0.684     70000
weighted avg      0.744     0.689     0.684     70000

